# Little notebook to plot HNL constraints (mixing vs mass)

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from scipy import interpolate

from matplotlib import rc, rcParams
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from matplotlib.pyplot import *

In [ ]:
def get_T2K_flux(mode="FHC",parent=pdg.pion,flavor=pdg.numu):
    
    if parent == pdg.pion:
        p = 'pion'
    elif parent == pdg.kaon:
        p = 'kaon'
    else:
        print(f'PDG={parent} parent mesons not supported.')
    
    e,f = np.genfr
    omtxt("T2K_flux_parent_meson/nd280_FHC_{}_")